## Use of global constant environment

Global constants can be used by nodes.
This is the main mechanism of providing environment
parameters to the graph at the runtime stage.

More generaly user can also use global constant environment, for instance
``import`` statements.

**The crucial point here is that it should be treated as constant, i.e. nodes must not modify or write to the environment. Unfortunately python is a flexible language and does not forbit this behaiviour thus it is in users' responsibilities.**

### Proper use

Let us take a look at the syntax in this simple example.
It's always a good practice to capitalize globals to separate from local variables.

In [1]:
from deltalanguage.lib import make_state_saver
from deltalanguage.runtime import DeltaPySimulator
from deltalanguage.wiring import DeltaGraph, Interactive


s = make_state_saver(int, verbose=True)


# Graph stage: connect nodes in a graph
# Example node that uses GLOBAL_PAR, which is not defined locally
@Interactive({}, int)
def foo(node):
    node.send(GLOBAL_PAR)

with DeltaGraph() as graph:
    s.save_and_exit(foo.call())


# Running stage: only now GLOBAL_PAR is defined
GLOBAL_PAR = 5
rt = DeltaPySimulator(graph)
rt.run()


# Just a check
assert s.saved[-1] == 5

saving 5


### Dangerous use, do not repeat this at home

Python provides a mechanism to turn global constants to global variables by using ``global``.

Look at this example:

In [2]:
# Graph stage:
@Interactive({}, int)
def bar(node):
    global GLOBAL_PAR
    GLOBAL_PAR += 5
    node.send(GLOBAL_PAR)


with DeltaGraph() as graph_bad:
    s.save_and_exit(bar.call())


# Running stage: only now GLOBAL_PAR is defined
GLOBAL_PAR = 5
rt = DeltaPySimulator(graph_bad)
rt.run()


# Just a check
assert s.saved[-1] == 10

saving 10


While it might be handy in procedural programming to change global variables,
it opens a dangerous door to side effects.
This is especially dangerous in the Deltaflow programming paradigm,
which operates with asyncrounous processes by default.
Thus if multiple nodes modify the same global environment
the result becomes non-deterministic!

In the example above, imagine the same graph is ran again in the same environment. Run the next cell multiple times:

In [3]:
rt = DeltaPySimulator(graph_bad)
rt.run()

assert s.saved[-1] != 10

saving 15


You see, the result changes due to environment even though the funtionality of nodes is the same.

**We discourage users from modifing global variables (i.e. they should be constants),
and more generaly changing the environment, from inside of a node.**

### Use case

Okay, now let's look at a more practical example with environment,
still simplistic though.

Say, some real experimental setup needs to be tested first,
and the number of iterations is a suitable candidate for a global constant.

In [4]:
# Graph stage: 
@Interactive({}, int)
def baz(node):
    for i in range(ITER_NUM):
        # do some hefty stuff
        print('FOO: ', i)
    node.send(ITER_NUM)


with DeltaGraph() as graph:
    s.save_and_exit(baz.call())


# Running stage: quick run for debugging
ITER_NUM = 3
rt_debug = DeltaPySimulator(graph)
rt_debug.run()


# Just a sanity check
assert s.saved[-1] == 3

FOO:  0
FOO:  1
FOO:  2
saving 3


Good, we've run a quick test and the program runs as expected on the simple debugging
runtime system.

The next step is to use it on real hardware where a lot of things can go wrong,
but hopefully the most severe bugs have been picked up already.

With global constants there is no need to redefine the graph,
just plug it in the experiment runtime:

In [5]:
# Running stage: a much larger number of iterations
ITER_NUM = 10
rt_exp = DeltaPySimulator(graph)
rt_exp.run()


# Just a sanity check
assert s.saved[-1] == 10

FOO:  0
FOO:  1
FOO:  2
FOO:  3
FOO:  4
FOO:  5
FOO:  6
FOO:  7
FOO:  8
FOO:  9
saving 10


This second run can be done on completely different hardware,
ans since ``DeltaGraph`` is hardware agnostic the Graph stage need not to be redone,
thus it should reduce the number of possible bugs.